In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from ipywidgets import widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import warnings
warnings.filterwarnings("ignore")

In [55]:
class eclairys:
    def __init__(self, stock_ticker, sector_ticker, market_ticker, short_period=[5, 10, 20], long_period=[50, 100, 200]):
        self.stock_ticker = stock_ticker
        self.sector_ticker = sector_ticker
        self.market_ticker = market_ticker
        self.short_period = short_period
        self.long_period = long_period
        self.period_list = short_period + long_period
        self.stock_history = None
        self.market_history = None
        self.sector_history = None
        self.market_relative_history = None
        self.sector_relative_history = None

    def fetch_data(self):
        """Fetch historical data for the stock and market."""
        self.stock_history = yf.Ticker(self.stock_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.stock_history.name = self.stock_ticker
        self.market_history = yf.Ticker(self.market_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.market_relative_history = self.stock_history / self.market_history
        self.sector_history = yf.Ticker(self.sector_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.sector_relative_history = self.stock_history / self.sector_history

    def calculate_moving_averages(self, data):
        """Calculate moving averages and moving average differentials."""
        MA_df, MAD_df = pd.DataFrame(), pd.DataFrame()
        for period in self.period_list:
            if len(data) < period:
                MA_df[f'MA {period}'] = np.nan
                MAD_df[f'MAD {period}'] = np.nan
            else:
                MA = data.rolling(period).mean()
                MA_df[f'MA {period}'] = MA
                MAD = MA.diff(5)
                MAD_df[f'MAD {period}'] = MAD
        return MA_df, MAD_df

    def normalize_mad(self, MAD_df):
        """Normalize MAD using a rolling window."""
        MADN_df = pd.DataFrame(index=MAD_df.index)
        rolling_max = MAD_df.rolling(200, min_periods=1).max()
        rolling_min = MAD_df.rolling(200, min_periods=1).min()
        MADN_df = 100 * (MAD_df - rolling_min) / (rolling_max - rolling_min)
        MADN_df = MADN_df.replace([np.inf, -np.inf], 0).fillna(0)
        return MADN_df

    def calculate_grade(self, MADN_df):
        """Calculate the GRADE score as the mean of normalized MAD values."""
        GRADE = MADN_df.mean(axis=1)
        return GRADE

    def generate_gps(self, MAD_df, short_period, long_period):
        """Generate GPS indicators based on booleanized MAD values."""
        BMAD = (MAD_df >= 0).astype(int)
        
        # Créer les noms de colonnes pour les périodes courtes et longues
        short_columns = [f'MAD {p}' for p in short_period]
        long_columns = [f'MAD {p}' for p in long_period]
        
        # Calculer la somme des périodes courtes et longues
        SBMAD = BMAD[short_columns].sum(axis=1)
        LBMAD = BMAD[long_columns].sum(axis=1)

        # Générer les indicateurs GPS en fonction des conditions
        conditions = [
            (SBMAD >= 2) & (LBMAD >= 2),
            (SBMAD < 2) & (LBMAD < 2),
            (SBMAD >= 2) & (LBMAD < 2),
            (SBMAD < 2) & (LBMAD >= 2)
        ]
        choices = ['A', 'B', 'C', 'P']
        GPS = pd.Series(np.select(conditions, choices, default=None), index=MAD_df.index)
        return GPS

    def run_analysis(self):
        """Run the entire analysis and return the output DataFrame."""
        self.fetch_data()
        
        # Calculate indicators for the stock
        stock_MA_df, stock_MAD_df = self.calculate_moving_averages(self.stock_history)
        stock_MADN_df = self.normalize_mad(stock_MAD_df)
        stock_GRADE = self.calculate_grade(stock_MADN_df)
        stock_GPS = self.generate_gps(stock_MAD_df, self.short_period, self.long_period)

        # Combine stock indicators into a DataFrame
        stock_output = pd.concat([
            pd.DataFrame(self.stock_history),
            stock_MA_df,
            pd.DataFrame(stock_GRADE, columns=['GRADE']),
            pd.DataFrame(stock_GPS, columns=['Absolute GPS'])
        ], axis=1)

        # Calculate indicators for the relative trend on sector
        sector_MA_df, sector_MAD_df = self.calculate_moving_averages(self.sector_relative_history)
        sector_GPS = self.generate_gps(sector_MAD_df, self.short_period, self.long_period)
        stock_output['Sector GPS'] = sector_GPS

        # Calculate indicators for the relative trend on market
        market_MA_df, market_MAD_df = self.calculate_moving_averages(self.market_relative_history)
        market_GPS = self.generate_gps(market_MAD_df, self.short_period, self.long_period)
        stock_output['Market GPS'] = market_GPS

        return stock_output

In [3]:
# Exemple d'utilisation de eclairys v4
eclairys('NVDA', '^IXIC', '^GSPC').run_analysis()

,NVDA,MA 5,MA 10,MA 20,MA 50,MA 100,MA 200,GRADE,Absolute GPS,Sector GPS,Market GPS
Date,,,,,,,,,,,
1999-01-22,0.037618,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1999-01-25,0.041559,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1999-01-26,0.038334,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1999-01-27,0.038215,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1999-01-28,0.038095,0.038764,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,146.270004,147.266000,142.606000,141.545499,129.392674,124.421128,108.057404,65.527458,A,A,A
2024-11-14,146.759995,146.841998,144.006000,142.037000,130.183857,124.627936,108.483635,64.845009,A,A,A
2024-11-15,141.979996,145.711996,144.664000,142.235999,130.967033,124.783844,108.878468,63.068669,A,A,A


In [4]:
# Function to fetch and process data
def fetch_data(stock_ticker, sector_ticker, market_ticker):
    df = eclairys(stock_ticker, sector_ticker, market_ticker).run_analysis()
    df = df.iloc[-200:]
    df = df.resample('D').ffill()
    return df

# Function to plot the data
def plot_data(stock_ticker, sector_ticker, market_ticker, df):
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(14, 12), sharex=True, gridspec_kw={'height_ratios': [3, 1, 0.25, 0.25, 0.25], 'hspace': 0.2})

    # 1st plot: Stock Price and Moving Averages
    ax1.plot(df.index, df[stock_ticker], label=stock_ticker, color='black', linewidth=1)
    ax1.plot(df.index, df['MA 5'], label='MA 5', linestyle='--', color='blue', linewidth=0.85)
    ax1.plot(df.index, df['MA 10'], label='MA 10', linestyle='--', color='orange', linewidth=0.85)
    ax1.plot(df.index, df['MA 20'], label='MA 20', linestyle='--', color='green', linewidth=0.85)
    ax1.plot(df.index, df['MA 50'], label='MA 50', linestyle='--', color='red', linewidth=0.85)
    ax1.plot(df.index, df['MA 100'], label='MA 100', linestyle='--', color='purple', linewidth=0.85)
    ax1.plot(df.index, df['MA 200'], label='MA 200', linestyle='--', color='brown', linewidth=0.85)
    ax1.legend(loc='upper left')
    ax1.set_title(f'{stock_ticker} Price and Moving Averages')
    ax1.grid(True)

    # 2nd plot: GRADE
    ax2.plot(df.index, df['GRADE'], color='black')
    ax2.axhline(y=75, color='green', linewidth=0.75)
    ax2.axhline(y=25, color='red', linewidth=0.75)
    ax2.set_title('GRADE')
    ax2.grid(True)

    # 3rd plot: Absolute GPS with color bands
    gps_colors = {'A': 'green', 'B': 'red', 'P': 'blue', 'C': 'orange'}
    for grade, color in gps_colors.items():
        mask = df['Absolute GPS'] == grade
        ax3.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax3.get_xaxis_transform())
    ax3.set_title('Absolute GPS')
    ax3.set_yticks([])

    # 4th plot: Market GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['Sector GPS'] == grade
        ax4.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax4.get_xaxis_transform())
    ax4.set_title('Sector GPS')
    ax4.set_yticks([])

    # 5th plot: Market GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['Market GPS'] == grade
        ax5.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax5.get_xaxis_transform())
    ax5.set_title('Market GPS')
    ax5.set_yticks([])

    # Add GPS legend
    legend_elements = [Patch(facecolor='green', label='A', alpha=0.5),
                       Patch(facecolor='red', label='B', alpha=0.5),
                       Patch(facecolor='blue', label='P', alpha=0.5),
                       Patch(facecolor='orange', label='C', alpha=0.5)]
    ax3.legend(handles=legend_elements, loc='upper left', title='GPS Categories')

    plt.tight_layout()
    plt.show()

# Widgets for interaction
stock_ticker_widget = widgets.Text(value='BA', description='Stock:')
sector_ticker_widget = widgets.Text(value='^SP500-20', description='Sector:')
market_ticker_widget = widgets.Text(value='^GSPC', description='Market:')
run_button = widgets.Button(description='Run Analysis', button_style='success')

# Output widget to display the plot
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        stock_ticker = stock_ticker_widget.value
        sector_ticker = sector_ticker_widget.value
        market_ticker = market_ticker_widget.value
        df = fetch_data(stock_ticker, sector_ticker, market_ticker)
        plot_data(stock_ticker, sector_ticker, market_ticker, df)

# Bind the button click event
run_button.on_click(on_button_click)

# Display the widgets
display(stock_ticker_widget, sector_ticker_widget, market_ticker_widget, run_button, output)


Text(value='BA', description='Stock:')

Text(value='^SP500-20', description='Sector:')

Text(value='^GSPC', description='Market:')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Output()

In [44]:
ticker = 'CAT'
history = pd.read_csv(f'/Users/matteobernard/Documents/Data Science/Database/Market Index/S&P 500/0. history/{ticker}.csv', index_col='Date', parse_dates=True)['Close']

In [45]:
def calculate_grade(history, short_period=[5, 10, 20], long_period=[50, 100, 200]):
    period_list = short_period + long_period
    MA_df, MAD_df = pd.DataFrame(), pd.DataFrame()
    for period in period_list:
        if len(history) < period:
            MA_df[f'MA {period}'] = np.nan
            MAD_df[f'MAD {period}'] = np.nan
        else:
            MA = history.rolling(period).mean()
            MA_df[f'MA {period}'] = MA
            MAD = MA.diff(5)
            MAD_df[f'MAD {period}'] = MAD
    MAD_df = MAD_df.dropna(axis=0)        
        
    MADN_df = pd.DataFrame(index=MAD_df.index)
    rolling_max = MAD_df.rolling(200, min_periods=1).max()
    rolling_min = MAD_df.rolling(200, min_periods=1).min()
    MADN_df = 100 * (MAD_df - rolling_min) / (rolling_max - rolling_min)
    MADN_df = MADN_df.replace([np.inf, -np.inf], 0).fillna(0)
    GRADE_df = MADN_df.mean(axis=1)
    
    return GRADE_df

In [50]:
index_df

,Component,Sector
Ticker,,
VZ,Verizon Communications Inc.,Communication Services
T,AT&T Inc.,Communication Services
LYV,"Live Nation Entertainment, Inc.",Communication Services
WBD,"Warner Bros. Discovery, Inc. Series A",Communication Services
EA,Electronic Arts Inc.,Communication Services
...,...,...
SRE,Sempra,Utilities
AEE,Ameren Corporation,Utilities
DTE,DTE Energy Company,Utilities


In [74]:
from tqdm import tqdm
grade_df = pd.DataFrame()
index_df = pd.read_csv('/Users/matteobernard/Documents/Data Science/Database/Market Index/S&P 500/S&P 500 - 2024.csv', index_col='Ticker')

for ticker in tqdm(index_df.index[:75]):
    history = pd.read_csv(f'/Users/matteobernard/Documents/Data Science/Database/Market Index/S&P 500/0. history/{ticker}.csv', index_col='Date', parse_dates=True)['Close']
    grade_df[ticker] = calculate_grade(history)

100%|██████████| 75/75 [00:02<00:00, 30.58it/s]


In [75]:
grade_df.describe()

,VZ,T,LYV,WBD,EA,TTWO,DIS,NFLX,META,MTCH,...,AZO,BBY,TPR,NKE,DECK,RL,LULU,PEP,KO,MNST
count,10481.000000,10481.000000,4719.000000,4837.000000,8959.000000,6985.000000,10481.000000,5653.000000,3047.000000,8090.000000,...,8560.000000,10113.000000,6077.000000,10481.000000,7897.000000,6943.000000,4302.000000,10481.000000,10481.000000,9946.000000
mean,51.013714,52.671821,53.361407,52.159488,51.804089,52.704105,51.652469,53.762924,53.793170,51.801351,...,52.167345,52.839084,52.971508,53.406529,52.862024,51.575407,52.809338,52.768669,53.985935,52.390108
std,18.673888,18.908457,19.478633,19.707014,19.499097,19.717643,19.425777,20.205221,18.617123,19.607199,...,20.210756,19.444123,20.645833,19.569419,20.099526,19.490683,19.981924,19.019427,19.351777,19.320187
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37.388544,38.906904,40.235570,38.671830,38.017587,38.900353,38.154438,39.664359,41.017781,37.485246,...,37.897312,38.723773,37.895873,39.673202,37.722850,37.983131,38.077921,40.047294,40.517152,38.354832
50%,51.040301,52.982261,53.774948,51.137958,52.292405,53.699682,52.531938,53.944377,54.807590,52.753064,...,52.761428,52.859928,53.952846,54.693988,52.720320,51.855436,52.611167,54.037117,55.267959,51.925442
75%,64.650911,66.812443,67.634832,66.124734,65.418723,66.404946,65.364140,68.599693,66.926461,66.106082,...,67.133059,67.331102,69.088963,68.204700,68.514597,65.757150,67.832397,66.871507,68.435053,66.605804
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [76]:
grade_df.loc['Sector'] = index_df.Sector[:75]

In [72]:
grade_df.groupby('Sector', axis=1).mean()

ValueError: len(index) != len(labels)

In [73]:
grade_df

,VZ,T,LYV,WBD,EA,TTWO,DIS,NFLX,META,MTCH,...,BBY,TPR,NKE,DECK,RL,LULU,PEP,KO,MNST,Sector
Date,,,,,,,,,,,,,,,,,,,,,
1984-08-31,0.000000,0.000000,NaN,NaN,NaN,NaN,62.305070,NaN,NaN,NaN,...,NaN,NaN,75.452855,NaN,NaN,NaN,46.238102,60.044095,NaN,NaN
1984-09-03,83.333333,33.333333,NaN,NaN,NaN,NaN,62.680001,NaN,NaN,NaN,...,NaN,NaN,77.443421,NaN,NaN,NaN,43.902065,60.211115,NaN,NaN
1984-09-04,66.666667,33.333333,NaN,NaN,NaN,NaN,63.482901,NaN,NaN,NaN,...,NaN,NaN,78.943404,NaN,NaN,NaN,40.279635,59.366062,NaN,NaN
1984-09-05,47.068506,33.333333,NaN,NaN,NaN,NaN,64.067618,NaN,NaN,NaN,...,NaN,NaN,78.614769,NaN,NaN,NaN,34.346908,58.071746,NaN,NaN
1984-09-06,59.788274,48.584616,NaN,NaN,NaN,NaN,64.968679,NaN,NaN,NaN,...,NaN,NaN,77.587874,NaN,NaN,NaN,29.885972,58.380087,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,19.782230,71.967161,83.865865,56.594443,50.635703,58.565604,41.097961,65.178650,45.195060,55.738870,...,35.914377,63.244881,48.765963,25.552288,50.645477,77.108026,53.966963,18.154799,68.646667,NaN
2024-10-29,18.711509,75.466051,84.679512,56.591769,49.470563,58.528626,39.642507,62.453523,45.744165,53.654364,...,33.983816,70.109098,46.277416,31.249939,51.193514,76.823827,47.660102,14.247613,66.466891,NaN
2024-10-30,14.748800,71.076866,84.912554,59.591879,52.886998,57.574726,37.658409,60.814587,49.011238,50.972457,...,32.449478,75.151773,44.577312,38.020464,51.899521,77.200175,42.816392,14.080504,64.643393,NaN
